In [1]:
import pandas as pd
from IPython.display import display

3 = nav klikšķēsma, 2 = daļēja klikšķēsma, 3 = ir klikšķēsma

### 1. Load & clean data 

In [100]:
df_supervisor_marks = pd.read_excel("original/supervisor_data.xlsx")
df_my_marks = pd.read_excel("original/my_data.xlsx")
print("df len's", len(df_supervisor_marks), len(df_my_marks))

df len's 5000 5000


In [101]:
# duplicated titles
df_duplicated_titles = df_supervisor_marks[df_supervisor_marks['Title'].duplicated()]
print(len(df_duplicated_titles))
df_duplicated_titles

21


,Nr,Title,mark
51,52,Navaļnija atbalstītāju protestos aizturēti vai...,1
1045,1046,Kariņš Valdības sēdē uzsprāgst no dusmām : “Mē...,3
1068,1069,Ļoti nepatīkamas ziņas lielai daļai 200 Eiro p...,3
1182,1183,Krievijā cietuma administrācija draud sākt Nav...,1
1266,1267,Atklājušās ļoti baisas detaļas par iepriekš pa...,3
1490,1491,"Pēc svētkiem koalīcija, iespējams, apspriedīs ...",1
1493,1494,Pavļuts parakstījis rīkojumu par visu iedzīotā...,1
1591,1592,Šomēnes liela daļa iedzīvotāju saņems valsts s...,3
1769,1770,Revidenti: Valsts ir pārmaksājusi par Covid-19...,1
1996,1997,"Ostapenko ""French Open"" pirmajā kārtā zaudē Ke...",1


In [102]:
# check that every headline is marked
null_masks = df_supervisor_marks.isnull().any(axis=1)
df_supervisor_marks[null_masks]

,Nr,Title,mark


In [103]:
# check that every headline is marked
null_masks = df_my_marks.isnull().any(axis=1)
df_my_marks[null_masks]

,Nr,Title,mark


In [104]:
# Somehow theree were negative values, so make them positive
df_my_marks['mark'] = df_my_marks['mark'].abs()
df_supervisor_marks['mark'] = df_supervisor_marks['mark'].abs()
print('Done')

Done


In [105]:
# check that df_supervisor_marks contains only 1, 2 or 3
df_supervisor_marks.loc[(df_supervisor_marks['mark'] <= 0) | (df_supervisor_marks['mark'] > 3)]

,Nr,Title,mark


In [106]:
# check that df_my_marks contains only 1, 2 or 3
df_my_marks.loc[(df_my_marks['mark'] <= 0) | (df_my_marks['mark'] > 3)]

,Nr,Title,mark


### 2. Differences in marks

In [107]:
# find differences between my and supervisor marks
df_diff_marks = df_supervisor_marks.compare(df_my_marks).reset_index().droplevel(1, axis=1)
df_titles = df_supervisor_marks[['Nr','Title']].reset_index()
df_diff_marks_with_title = df_diff_marks.merge(df_titles, how='inner', on="index")
df_diff_marks_with_title = df_diff_marks_with_title.set_axis(['index',
                                                              'supervisor_mark',
                                                              'my_mark',
                                                              'Nr',
                                                              'Title'], axis=1)
df_diff_marks_with_title.to_excel("different_marks.xlsx") 
df_diff_marks_with_title

,index,supervisor_mark,my_mark,Nr,Title
0,1,2.0,3.0,2,Dramatiski skati Vācijā - siltā laika dēļ pils...
1,2,2.0,3.0,3,"LNSO iegādājas 30 tūkstoš eiro vērtu fagotu, n..."
2,5,1.0,3.0,6,Pētniece: Laimes hormoni rodas arī vēderā
3,8,1.0,2.0,9,Ambiciozs pieteikums stabilai nākotnei – studē...
4,14,1.0,3.0,15,Atjaunojamās enerģijas jomā pārliecinoši airēj...
...,...,...,...,...,...
1525,4990,1.0,2.0,4991,Rinkēvičs: No NATO tikšanās sagaidām arī konkr...
1526,4991,1.0,2.0,4992,Slimnīcu kapacitātes stiprināšanai piešķir vai...
1527,4994,1.0,3.0,4995,Šimkus: Vara skaidrāk kā jebkad atklāj ierobež...
1528,4995,2.0,1.0,4996,Bolīvijā ar krāšņu parādi sāk gatavoties Ziema...


In [16]:
# find 1 vs 3 marks
df_diff13 = df_diff_marks_with_title.loc[
    (df_diff_marks_with_title['supervisor_mark'] == 3) & (df_diff_marks_with_title['my_mark'] == 1)
    |
    (df_diff_marks_with_title['supervisor_mark'] == 1) & (df_diff_marks_with_title['my_mark'] == 3)
]
df_diff13.to_excel('differences_13.xlsx', index=False)
df_diff13

,index,supervisor_mark,my_mark,Title
2,5,1.0,3.0,Pētniece: Laimes hormoni rodas arī vēderā
4,14,1.0,3.0,Atjaunojamās enerģijas jomā pārliecinoši airēj...
10,43,1.0,3.0,Pret Jencīša jaungada pasākumā pieķerto tiesne...
15,59,1.0,3.0,Sūtījumu nozare aicina cilvēkus būt saprotošie...
18,66,1.0,3.0,«Freedom House»: Arvien vairāk valstu izmanto ...
...,...,...,...,...
1514,4947,1.0,3.0,Dienvidāfrikas ārste: Cilvēki ar Covid-19 jaun...
1517,4959,1.0,3.0,Kā ir aklam bērnam mācīties parastā skolā? Stā...
1518,4963,1.0,3.0,Bricis jau laicīgi brīdina iedzīvotājus : “Šon...
1520,4972,3.0,1.0,Populāras spēles aukstiem ziemas vakariem


Kopa 1530 atšķirošo markējumu; 696 no tiem ir pilnība polārie (1-3 vai 3-1)

In [11]:
df_my_diff = df_diff_marks_with_title.loc[df_diff_marks_with_title['index'] >= 3016]
df_my_diff.to_excel('my_set_differences.xlsx', index=False)

In [12]:
df_prev_st_diff = df_diff_marks_with_title.loc[df_diff_marks_with_title['index'] < 3016]
df_prev_st_diff.to_excel('prev_student_set_differences.xlsx', index=False)

In [18]:
print('my diff via supervisor-',len(df_my_diff))
print('prev student diff via supervisor-',len(df_prev_st_diff))
print('total mismatches-',len(df_diff_marks_with_title))
print('mismatches with delta 2 (1 vs 3 or 3 vs 1)-',len(df_diff13))

my diff via supervisor- 794
prev student diff via supervisor- 736
total mismatches- 1530
mismatches with delta 2 (1 vs 3 or 3 vs 1)- 696


### 3. Matched marks

In [108]:
# get matched marks
df_matched_nr = df_supervisor_marks[ ~df_titles.Nr.isin(df_diff_marks_with_title.Nr) ]
print('test (total matched + non matched= 5000):', len(df_matched_nr) + len(df_diff_marks_with_title) == 5000)
df_matched_nr.to_excel('match_by_Nr.xlsx', index=False)
df_matched_nr

test (total matched + non matched= 5000): True


,Nr,Title,mark
0,1,Trūkstošu izziņu par negatīviem Covid-19 testi...,1
3,4,Saslimstības riski ar Covid-19 joprojām ir aug...,1
4,5,"Atvieglojumus nesola, bet skaidrību uz ilgāku ...",3
6,7,Iereibuša vadītāja izraisītajā avārijā Zemgalē...,1
7,8,Cilvēkiem jau laikus iesaka gatavoties Trešaja...,1
...,...,...,...
4992,4993,Izgaismotajā Mežaparka estrādē skan Mārtiņa Br...,1
4993,4994,Latvijā no Baltkrievijas nelikumīgi mēģinājuši...,1
4996,4997,No 2022.gada 1.janvāra plānots samazināt OIK p...,1
4998,4999,Kuršu iniciatīvu fonds veidos podnieku darbnīc...,1


In [98]:
# matched headlines stats
df_matched_nr_stats = df_matched_nr.groupby(by=["mark"]).count()
df_matched_nr_stats['perc%'] = (df_matched_nr_stats['Title'] / len(df_matched_nr))*100
df_matched_nr_stats

,Nr,Title,perc%
mark,,,
1,2269,2269,65.389049
2,57,57,1.642651
3,1144,1144,32.968300


#### Play with GPT

In [44]:
# Concatinate all diffs with gpt marks
df_gpt_prev_student_mark = pd.read_excel("prev_student_set_diffs_labeled_gpt.xlsx")
df_gpt_my_mark = pd.read_excel("my_set_differences_labeled_gpt.xlsx")
df_diffs_with_gpt = pd.concat([df_gpt_prev_student_mark, df_gpt_my_mark])
df_diffs_with_gpt.to_excel('different_marks_total_with_gpt.xlsx', index=False)
df_diffs_with_gpt

,index,supervisor_mark,my_mark,Title,gpt_mark
0,1,2,3,Dramatiski skati Vācijā - siltā laika dēļ pils...,3
1,2,2,3,"LNSO iegādājas 30 tūkstoš eiro vērtu fagotu, n...",2
2,5,1,3,Pētniece: Laimes hormoni rodas arī vēderā,3
3,8,1,2,Ambiciozs pieteikums stabilai nākotnei – studē...,2
4,14,1,3,Atjaunojamās enerģijas jomā pārliecinoši airēj...,1
...,...,...,...,...,...
789,4990,1,2,Rinkēvičs: No NATO tikšanās sagaidām arī konkr...,1
790,4991,1,2,Slimnīcu kapacitātes stiprināšanai piešķir vai...,1
791,4994,1,3,Šimkus: Vara skaidrāk kā jebkad atklāj ierobež...,1
792,4995,2,1,Bolīvijā ar krāšņu parādi sāk gatavoties Ziema...,1


In [48]:
df_gpt_my_superv_diff = df_diffs_with_gpt.loc[~(df_diffs_with_gpt['gpt_mark'] == df_diffs_with_gpt['supervisor_mark']) 
                      & 
                      ~(df_diffs_with_gpt['gpt_mark'] == df_diffs_with_gpt['my_mark'])
                     ]
df_gpt_my_superv_diff

,index,supervisor_mark,my_mark,Title,gpt_mark
5,15,2,3,Pārbaude par Covid-19 vakcīnu iegādi: Pavļuts ...,1
6,17,2,3,"Partijai ""KPV LV"" reģistrēts jauns nosaukums u...",1
7,23,2,3,'Nē' maznozīmīgiem ceļojumiem un piespiedu izo...,1
8,26,2,3,"Ābolu pīrāgs ar kefīru, kas burtiski kūst – vi...",1
11,46,2,3,Tagad vairs nebūs aizliegto preču – Tā vietā B...,1
...,...,...,...,...,...
777,4942,2,3,Ko Tev sola zvaigznes? Horoskopi līdz 2.decembrim,1
779,4955,2,3,Neatkarīgā iesaka: TOP 5 bezmaksas Pirmās adve...,1
780,4957,2,3,Mājīgās sveču liesmiņas var kaitēt senioriem u...,1
785,4974,3,2,Apsver arī Latvijā noteikt vakcinācijas sertif...,1


In [53]:
df_gpt_my_superv_diff.loc[df_gpt_my_superv_diff['index'] >= 3016]

,index,supervisor_mark,my_mark,Title,gpt_mark
8,3051,2,3,"Nu jau oficiāli! Valdība paziņojusi nozares, k...",1
14,3072,2,3,Krievijā no daudzstāvu ēkas logiem laužas ārā ...,1
30,3101,2,3,Kā uztaisīt odu ķērāju 5 minūtēs pašam,1
39,3119,2,3,Vai ir jāatstāj tējas nauda restorānā un cik d...,1
40,3122,2,3,Ir ticis paziņots kāds varetu būt rīcības plān...,1
...,...,...,...,...,...
777,4942,2,3,Ko Tev sola zvaigznes? Horoskopi līdz 2.decembrim,1
779,4955,2,3,Neatkarīgā iesaka: TOP 5 bezmaksas Pirmās adve...,1
780,4957,2,3,Mājīgās sveču liesmiņas var kaitēt senioriem u...,1
785,4974,3,2,Apsver arī Latvijā noteikt vakcinācijas sertif...,1


In [54]:
df.loc[df['gpt_mark'] != 1]

,index,supervisor_mark,my_mark,Title,gpt_mark
131,3286,1,3,AIP pauž bažas par stipediju apmēru,2
244,3538,1,2,Izplatās viltus ziņa par piespiedu Covid-19 te...,3
282,3622,1,2,Traģēdija Rīgas būvlaukumā: smagās tehnikas va...,3
305,3688,1,2,Klimata pārmaiņas var piespiest migrēt līdz 21...,3
512,4168,1,3,"20 Eiro mēnesī vakcinētajiem, bet ne visiem! U...",2


## Analysis after re-marking

In [3]:
pip install glob2

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import glob2 as glob
import pandas as pd

### Load re-marked excels

In [119]:
# read all excels marked by several people into one dataframe
diff_remark_files = glob.glob("differences_re-marked" + "/*.xlsx")
li = []
for filename in diff_remark_files:
    df = pd.read_excel(filename, index_col=None, header=0)
    li.append(df)
    
diff_df = pd.concat(li, axis=0, ignore_index=True)
diff_df

,index,supervisor_mark,student_mark,third_part_mark,Title
0,1,2,3,2,Dramatiski skati Vācijā - siltā laika dēļ pils...
1,2,2,3,2,"LNSO iegādājas 30 tūkstoš eiro vērtu fagotu, n..."
2,5,1,3,1,Pētniece: Laimes hormoni rodas arī vēderā
3,8,1,2,2,Ambiciozs pieteikums stabilai nākotnei – studē...
4,14,1,3,1,Atjaunojamās enerģijas jomā pārliecinoši airēj...
...,...,...,...,...,...
1525,4602,1,2,3,"""airBaltic"" lidmašīna atgriežas Tallinas lidos..."
1526,4609,3,2,3,Ziema tuvojas! “Cilvēkus brīdina par nākamās n...
1527,4612,3,2,3,Daugavpils gāzes darījums raisa jautājumus
1528,4617,1,2,2,Sarunu festivāla direktore: Diskusijās par vak...


In [120]:
# check that df contains only 1, 2 or 3
print(diff_df.loc[(diff_df['supervisor_mark'] <= 0) | (diff_df['supervisor_mark'] > 3)])
print(diff_df.loc[(diff_df['student_mark'] <= 0) | (diff_df['student_mark'] > 3)])
print(diff_df.loc[(diff_df['third_part_mark'] <= 0) | (diff_df['third_part_mark'] > 3)])

Empty DataFrame
Columns: [index, supervisor_mark, student_mark, third_part_mark, Title]
Index: []
Empty DataFrame
Columns: [index, supervisor_mark, student_mark, third_part_mark, Title]
Index: []
Empty DataFrame
Columns: [index, supervisor_mark, student_mark, third_part_mark, Title]
Index: []


In [121]:
# check that every headline is marked
null_marks = diff_df.isnull().any(axis=1)
diff_df[null_marks]

,index,supervisor_mark,student_mark,third_part_mark,Title


### Find resulting mark

In [122]:
# find the most frequent mark (mode of row) and concat with diff_df
mode_df = diff_df[
    ["supervisor_mark","student_mark","third_part_mark"]
].mode(axis='columns', numeric_only=True)
mode_df.columns = ['mode_mark', '2_mode', '3_mode']

diff_df_with_mode = pd.concat([diff_df, mode_df], axis=1)
diff_df_with_mode

,index,supervisor_mark,student_mark,third_part_mark,Title,mode_mark,2_mode,3_mode
0,1,2,3,2,Dramatiski skati Vācijā - siltā laika dēļ pils...,2.0,NaN,NaN
1,2,2,3,2,"LNSO iegādājas 30 tūkstoš eiro vērtu fagotu, n...",2.0,NaN,NaN
2,5,1,3,1,Pētniece: Laimes hormoni rodas arī vēderā,1.0,NaN,NaN
3,8,1,2,2,Ambiciozs pieteikums stabilai nākotnei – studē...,2.0,NaN,NaN
4,14,1,3,1,Atjaunojamās enerģijas jomā pārliecinoši airēj...,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...
1525,4602,1,2,3,"""airBaltic"" lidmašīna atgriežas Tallinas lidos...",1.0,2.0,3.0
1526,4609,3,2,3,Ziema tuvojas! “Cilvēkus brīdina par nākamās n...,3.0,NaN,NaN
1527,4612,3,2,3,Daugavpils gāzes darījums raisa jautājumus,3.0,NaN,NaN
1528,4617,1,2,2,Sarunu festivāla direktore: Diskusijās par vak...,2.0,NaN,NaN


In [123]:
# find rows where all three classes exist
still_no_max_df = diff_df_with_mode[~diff_df_with_mode.isnull().any(axis=1)]
still_no_max_df

,index,supervisor_mark,student_mark,third_part_mark,Title,mode_mark,2_mode,3_mode
18,66,1,3,2,«Freedom House»: Arvien vairāk valstu izmanto ...,1.0,2.0,3.0
30,104,3,1,2,Pēc baznīcas kristību rituāla Rumānijā mirst 6...,1.0,2.0,3.0
38,144,1,3,2,"Angīna: Kā to izārstēt īsā laikā, mājas apstāk...",1.0,2.0,3.0
40,151,1,3,2,Elvi akcijas – vienmēr pirktākajām precēm,1.0,2.0,3.0
44,163,1,3,2,Pie veikaliem drūzma un rindas! Stājas spēkā p...,1.0,2.0,3.0
...,...,...,...,...,...,...,...,...
1509,4552,2,3,1,Kā sākt spēlēt azartspēles internetā?,1.0,2.0,3.0
1519,4584,1,3,2,"Ansis Klintsons izslimojis Covid-19 un saka: ""...",1.0,2.0,3.0
1520,4586,1,2,3,Putins sola Baltkrievijai atbalstu pret ārvals...,1.0,2.0,3.0
1524,4599,1,3,2,RAKUS katastrofāli trūkst resursu; Covid-19 pa...,1.0,2.0,3.0


In [124]:
# rows with existing dominant class
res_mark_df = diff_df_with_mode[diff_df_with_mode.isnull().any(axis=1)].drop(['2_mode', '3_mode'], axis=1)
res_mark_df

,index,supervisor_mark,student_mark,third_part_mark,Title,mode_mark
0,1,2,3,2,Dramatiski skati Vācijā - siltā laika dēļ pils...,2.0
1,2,2,3,2,"LNSO iegādājas 30 tūkstoš eiro vērtu fagotu, n...",2.0
2,5,1,3,1,Pētniece: Laimes hormoni rodas arī vēderā,1.0
3,8,1,2,2,Ambiciozs pieteikums stabilai nākotnei – studē...,2.0
4,14,1,3,1,Atjaunojamās enerģijas jomā pārliecinoši airēj...,1.0
...,...,...,...,...,...,...
1523,4591,2,3,2,Mediķi un politiķi vērtē Pavļuta atbildību par...,2.0
1526,4609,3,2,3,Ziema tuvojas! “Cilvēkus brīdina par nākamās n...,3.0
1527,4612,3,2,3,Daugavpils gāzes darījums raisa jautājumus,3.0
1528,4617,1,2,2,Sarunu festivāla direktore: Diskusijās par vak...,2.0


In [125]:
# stats
res_mark_df_stats = res_mark_df.groupby(by=["mode_mark"]).count()[['Title']]
res_mark_df_stats['perc%'] = (res_mark_df_stats['Title'] / len(res_mark_df))*100
res_mark_df_stats

,Title,perc%
mode_mark,,
1.0,592,49.747899
2.0,357,30.000000
3.0,241,20.252101


In [126]:
# Assume that headlines that marked via 3 different class have class 2 as a mode mark.
# Concat with res_mark_df
midle_mark_df = still_no_max_df.copy().drop(['2_mode', '3_mode'], axis=1)
midle_mark_df["mode_mark"] = 2.0
res_mark_df = pd.concat([res_mark_df, midle_mark_df], axis=0)
res_mark_df

,index,supervisor_mark,student_mark,third_part_mark,Title,mode_mark
0,1,2,3,2,Dramatiski skati Vācijā - siltā laika dēļ pils...,2.0
1,2,2,3,2,"LNSO iegādājas 30 tūkstoš eiro vērtu fagotu, n...",2.0
2,5,1,3,1,Pētniece: Laimes hormoni rodas arī vēderā,1.0
3,8,1,2,2,Ambiciozs pieteikums stabilai nākotnei – studē...,2.0
4,14,1,3,1,Atjaunojamās enerģijas jomā pārliecinoši airēj...,1.0
...,...,...,...,...,...,...
1509,4552,2,3,1,Kā sākt spēlēt azartspēles internetā?,2.0
1519,4584,1,3,2,"Ansis Klintsons izslimojis Covid-19 un saka: ""...",2.0
1520,4586,1,2,3,Putins sola Baltkrievijai atbalstu pret ārvals...,2.0
1524,4599,1,3,2,RAKUS katastrofāli trūkst resursu; Covid-19 pa...,2.0


In [127]:
res_mark_df.to_excel('differences_re-marked/different_marks_total.xlsx', index=False)

### Combine to resulting marks

In [128]:
# Load original data
df_supervisor_marks = pd.read_excel("original/supervisor_data.xlsx")
df_my_marks = pd.read_excel("original/my_data.xlsx")
print("df len's", len(df_supervisor_marks), len(df_my_marks))

df len's 5000 5000


In [129]:
# Slice from original excel all other matched marks and add them to res_mark_df
res_mark_df["Nr"] = res_mark_df["index"] + 1
res_non_match_df = res_mark_df[['Nr', 'Title','mode_mark']]
res_non_match_df = res_non_match_df.rename({'mode_mark' : 'mark'}, axis='columns')
df_matched = df_supervisor_marks[ ~df_supervisor_marks.Nr.isin(res_non_match_df.Nr) ]
res_df = pd.concat([df_matched, res_non_match_df], axis=0)
res_df

,Nr,Title,mark
0,1,Trūkstošu izziņu par negatīviem Covid-19 testi...,1.0
3,4,Saslimstības riski ar Covid-19 joprojām ir aug...,1.0
4,5,"Atvieglojumus nesola, bet skaidrību uz ilgāku ...",3.0
6,7,Iereibuša vadītāja izraisītajā avārijā Zemgalē...,1.0
7,8,Cilvēkiem jau laikus iesaka gatavoties Trešaja...,1.0
...,...,...,...
1509,4553,Kā sākt spēlēt azartspēles internetā?,2.0
1519,4585,"Ansis Klintsons izslimojis Covid-19 un saka: ""...",2.0
1520,4587,Putins sola Baltkrievijai atbalstu pret ārvals...,2.0
1524,4600,RAKUS katastrofāli trūkst resursu; Covid-19 pa...,2.0


In [135]:
res_df = res_df.sort_values(by=['Nr'])
res_df.to_excel('marked_headlines.xlsx', index=False)

In [136]:
# checks 
print('Headlines count = ', len(res_df), len(res_df) == 5000)
print('Unique Nr =  ', len(res_df['Nr'].unique()), len(res_df['Nr'].unique()) == 5000)
null_marks = res_df.isnull().any(axis=1)
print('Non marked headlines = ', len(res_df[null_marks]))
print('Marks NOT in range [1, 2, 3] = ', 
      len(res_df.loc[(res_df['mark'] <= 0) | (res_df['mark'] > 3)]))

Headlines count =  5000 True
Unique Nr =   5000 True
Non marked headlines =  0
Marks NOT in range [1, 2, 3] =  0


In [137]:
# stats
res_df_stats = res_df.groupby(by=["mark"]).count()[['Title']]
res_df_stats['perc%'] = (res_df_stats['Title'] / len(res_df))*100
res_df_stats

,Title,perc%
mark,,
1.0,2861,57.22
2.0,754,15.08
3.0,1385,27.70


In [139]:
754/(754 + 1385)

0.3525011687704535